In [1]:
import gzip
import random
import scipy
from collections import defaultdict
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
import csv
import numpy as np
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import BaselineOnly
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import SVDpp
from surprise import NMF
from surprise import accuracy
from surprise.model_selection import KFold

In [2]:
dataDir = "C:/Users/云舒/21Fall/CSE258/HW3/"

In [3]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

In [4]:
allRatings = []
userRatings = defaultdict(list)
userIDs = []
itemIDs = []

for user,recipe,d in readCSV("trainInteractions.csv.gz"):
    userIDs.append(user)
    itemIDs.append(recipe)
    r = int(d['rating'])
    allRatings.append(r)
    userRatings[user].append(r)

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
for u in userRatings:
  userAverage[u] = sum(userRatings[u]) / len(userRatings[u])


In [5]:
allRatings = []
userRatings = defaultdict(list)
itemRatings = defaultdict(list)

for user,recipe,d in readCSV("trainInteractions.csv.gz"):
    r = int(d['rating'])
    allRatings.append(r)
    userRatings[user].append(r)
    itemRatings[recipe].append(r)

globalAverage = sum(allRatings) / len(allRatings)

userAverage = {}
itemAverage = {}
for u in userRatings:
    userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
for i in itemRatings:
    itemAverage[i] = sum(itemRatings[i])/len(itemRatings[i])

In [6]:
reader = Reader(line_format='user item timestamp rating', sep=',',skip_lines=1)
data = Dataset.load_from_file(dataDir + "trainInteractions.csv", reader=reader)
trainset, testset = train_test_split(data, test_size=.25)
model = SVD(n_epochs = 10,lr_all= 0.005,reg_all = 0.1)
model.fit(trainset)

In [7]:
trainset, testset = train_test_split(data, test_size=.25)

In [ ]:
n_epochs = [5,10,15,20]
n_factors = [50,100,150]
lr_all = [0.0001,0.001,0.01,0.1,0.2,0.3]
reg_all = [0.4,0.1,0.05]

In [14]:
param_grid = {'n_epochs': [5,10,15,20], 
              'n_factors':[20,25]
              'lr_all': [0.004,0.005],
              'reg_all': [0.2,0.1,0.05]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
 
gs.fit(data)


In [15]:
print(gs.best_params['rmse'])

{'n_epochs': 15, 'reg_all': 0.1}


In [16]:
model = SVD(n_epochs = 10,lr_all= 0.005,reg_all = 0.1)
model.fit(trainset)

In [21]:
print(gs.best_params['rmse'])

{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [40]:

reader = Reader(line_format='user item timestamp rating', sep=',',skip_lines=1)
data = Dataset.load_from_file("trainInteractions.csv", reader=reader)
trainset, testset = train_test_split(data, test_size=.1)
# lr_all: 0.004
# nfactor: 40
# for i in range(20,35):
while True:
    model3 = SVD(n_factors=25, n_epochs=20, reg_all=0.2, lr_all=0.004)
    model3.fit(trainset)
    predictions = model3.test(testset)

    sse = 0
    for p in predictions:
        sse += (p.r_ui - p.est)**2
    print(sse / len(predictions))
    if sse / len(predictions) < 0.81:
        break

In [45]:
'62194177' in userIDs

True

In [17]:
predictions = open("predictions_Rated.txt", 'w')
for l in open("stub_Rated.txt"):
    if l.startswith("user_id"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if u in userIDs and i in itemIDs:
        pred = model.predict(u,i).est
    elif u in userIDs:
        pred = userAverage[u]
    elif i in itemIDs:
        pred = itemAverage[i]
    else:
        pred = globalAverage
    predictions.write(u + '-' + i + ',' + str(pred) + '\n')

predictions.close()

In [39]:
model.predict('62194177','56272925').est

4.8218405288402435

## gogogo

In [51]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
 
gs.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'split'

In [10]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
algo = NormalPredictor()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm NormalPredictor on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.1916  1.1906  1.1922  1.1915  0.0007  
MAE (testset)     0.7824  0.7827  0.7820  0.7824  0.0003  
Fit time          0.58    0.75    0.73    0.69    0.07    
Test time         1.54    1.86    1.69    1.69    0.13    


{'test_rmse': array([1.19163827, 1.19055288, 1.19223396]),
 'test_mae': array([0.78236409, 0.78270049, 0.7820076 ]),
 'fit_time': (0.583441972732544, 0.7461717128753662, 0.7310459613800049),
 'test_time': (1.5358905792236328, 1.856036901473999, 1.6888036727905273)}

### 使用均值协同过滤

In [8]:
algo = KNNWithMeans()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9844  0.9788  0.9809  0.9814  0.0023  
MAE (testset)     0.5698  0.5673  0.5682  0.5684  0.0010  
Fit time          8.61    10.94   9.54    9.70    0.96    
Test time         6.20    6.74    5.99    6.31    0.32    


{'test_rmse': array([0.98443284, 0.97879351, 0.98094189]),
 'test_mae': array([0.56981364, 0.56731326, 0.5682112 ]),
 'fit_time': (8.60611605644226, 10.944492816925049, 9.542875528335571),
 'test_time': (6.197307825088501, 6.743747234344482, 5.9947521686553955)}

In [9]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse', 'mae'], cv=3)
 
gs.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [10]:
print(gs.best_params['rmse'])

{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}


In [11]:
model = KNNWithMeans(n_epochs = 5,lr_all= 0.002,reg_all = 0.4)
model.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [12]:
predictions = open("predictions_Rated.txt", 'w')
for l in open("stub_Rated.txt"):
    if l.startswith("user_id"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if u in userIDs and i in itemIDs:
        pred = model.predict(u,i).est
    elif u in userIDs:
        pred = userAverage[u]
    elif i in itemIDs:
        pred = itemAverage[i]
    else:
        pred = globalAverage
    predictions.write(u + '-' + i + ',' + str(pred) + '\n')

predictions.close()

### NMF

In [13]:
algo = NMF()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

ZeroDivisionError: float division